In [0]:
#Hello
import pandas as pd
import string
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn_pandas import DataFrameMapper
from xgboost import XGBClassifier
from sklearn import preprocessing

In [0]:
df = pd.read_csv('train.csv', index_col="Loan_ID")

In [27]:
df.head()

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
Loan_ID,,,,,,,,,,,,,
10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [28]:
df.isna().sum()

Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

#### Handling missing data

In [0]:
def cleaning_data(df):
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].str.replace(",","")
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].astype(float)
    df["Length_Employed"] = df["Length_Employed"].str.replace('< 1 year','0')
    df["Length_Employed"] = df["Length_Employed"].str.replace(r'[><+]+',"")
    df["Length_Employed"] = df["Length_Employed"].str.extract('(\d+)').astype(float)
    df["Length_Employed"] = df["Length_Employed"].transform(lambda x: x.fillna(-99999))
    df["Annual_Income"] = df.groupby(["Length_Employed"])["Annual_Income"].transform(lambda x: x.fillna(x.median()))
    #df["Home_Owner"] = df["Home_Owner"].transform(lambda x: x.fillna("-1"))
    df["Months_Since_Deliquency"] = df["Months_Since_Deliquency"].transform(lambda x: x.fillna(-1))
    return df

In [0]:
df = cleaning_data(df)

In [0]:
from scipy import stats
def normalize_data(df):
    scaler = MinMaxScaler()
    df["Loan_Amount_Requested"] = scaler.fit_transform(df[["Loan_Amount_Requested"]])
    df["Length_Employed"] = scaler.fit_transform(df[["Length_Employed"]])
    df["Annual_Income"] = scaler.fit_transform(df[["Annual_Income"]])
    df["Months_Since_Deliquency"] = scaler.fit_transform(df[["Months_Since_Deliquency"]])
    df["Debt_To_Income"] = scaler.fit_transform(df[["Debt_To_Income"]])
    df["Number_Open_Accounts"] = scaler.fit_transform(df[["Number_Open_Accounts"]])
    df["Total_Accounts"] = scaler.fit_transform(df[["Total_Accounts"]])
    df["Inquiries_Last_6Mo"] = scaler.fit_transform(df[["Inquiries_Last_6Mo"]])
    return df

In [0]:
original_df = df.copy()
df = df.drop(["Interest_Rate"], axis=1)

In [0]:
scaled_features_df = normalize_data(df)

In [34]:
scaled_features_df

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
Loan_ID,,,,,,,,,,,,
10000001,0.188406,0.99990,Rent,0.008538,not verified,car,0.459365,0.000,0.000000,0.118421,0.077922,Female
10000002,0.855072,0.99994,Mortgage,0.007471,VERIFIED - income,debt_consolidation,0.373343,0.000,0.099448,0.157895,0.142857,Female
10000003,0.702174,0.99997,Mortgage,0.009547,VERIFIED - income source,debt_consolidation,0.397099,0.000,0.000000,0.157895,0.090909,Male
10000004,0.449275,0.99990,NaN,0.006958,VERIFIED - income source,debt_consolidation,0.358590,0.375,0.000000,0.210526,0.129870,Male
10000005,0.478261,0.99998,Own,0.012273,VERIFIED - income source,debt_consolidation,0.554389,0.125,0.000000,0.250000,0.181818,Female
...,...,...,...,...,...,...,...,...,...,...,...,...
10164305,0.586232,1.00000,Rent,0.008805,VERIFIED - income source,credit_card,0.339085,0.000,0.000000,0.171053,0.090909,Male
10164306,0.159420,0.99998,Own,0.007871,VERIFIED - income source,small_business,0.228057,0.000,0.204420,0.171053,0.181818,Male
10164307,0.572464,1.00000,Mortgage,0.008805,VERIFIED - income,credit_card,0.454364,0.125,0.000000,0.223684,0.181818,Male


In [0]:
def label_encode(df):
    onehotlabels = pd.get_dummies(df, prefix=None, prefix_sep='_', 
                       dummy_na=True, columns=["Income_Verified","Home_Owner","Purpose_Of_Loan","Gender"], sparse=False, 
                       drop_first=False, dtype=None)
    return onehotlabels

In [0]:
scaled_features_df = label_encode(scaled_features_df)

In [37]:
scaled_features_df

,Loan_Amount_Requested,Length_Employed,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Income_Verified_VERIFIED - income,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Income_Verified_nan,Home_Owner_Mortgage,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Home_Owner_nan,Purpose_Of_Loan_car,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Purpose_Of_Loan_nan,Gender_Female,Gender_Male,Gender_nan
Loan_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000001,0.188406,0.99990,0.008538,0.459365,0.000,0.000000,0.118421,0.077922,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10000002,0.855072,0.99994,0.007471,0.373343,0.000,0.099448,0.157895,0.142857,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10000003,0.702174,0.99997,0.009547,0.397099,0.000,0.000000,0.157895,0.090909,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10000004,0.449275,0.99990,0.006958,0.358590,0.375,0.000000,0.210526,0.129870,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10000005,0.478261,0.99998,0.012273,0.554389,0.125,0.000000,0.250000,0.181818,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10164305,0.586232,1.00000,0.008805,0.339085,0.000,0.000000,0.171053,0.090909,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10164306,0.159420,0.99998,0.007871,0.228057,0.000,0.204420,0.171053,0.181818,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
10164307,0.572464,1.00000,0.008805,0.454364,0.125,0.000000,0.223684,0.181818,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
X_train = scaled_features_df

In [39]:
X_train

,Loan_Amount_Requested,Length_Employed,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Income_Verified_VERIFIED - income,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Income_Verified_nan,Home_Owner_Mortgage,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Home_Owner_nan,Purpose_Of_Loan_car,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Purpose_Of_Loan_nan,Gender_Female,Gender_Male,Gender_nan
Loan_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000001,0.188406,0.99990,0.008538,0.459365,0.000,0.000000,0.118421,0.077922,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10000002,0.855072,0.99994,0.007471,0.373343,0.000,0.099448,0.157895,0.142857,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10000003,0.702174,0.99997,0.009547,0.397099,0.000,0.000000,0.157895,0.090909,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10000004,0.449275,0.99990,0.006958,0.358590,0.375,0.000000,0.210526,0.129870,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10000005,0.478261,0.99998,0.012273,0.554389,0.125,0.000000,0.250000,0.181818,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10164305,0.586232,1.00000,0.008805,0.339085,0.000,0.000000,0.171053,0.090909,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10164306,0.159420,0.99998,0.007871,0.228057,0.000,0.204420,0.171053,0.181818,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
10164307,0.572464,1.00000,0.008805,0.454364,0.125,0.000000,0.223684,0.181818,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
y_train = original_df["Interest_Rate"]

In [41]:
y_train

Loan_ID
10000001    1
10000002    3
10000003    3
10000004    3
10000005    1
           ..
10164305    2
10164306    2
10164307    1
10164308    3
10164309    2
Name: Interest_Rate, Length: 164309, dtype: int64

In [0]:
# A parameter grid for XGBoost
random_grid = {
        'booster' : ['gbtree','dart'],
        'n_estimators' : [300, 350, 500, 650, 800],
        'max_depth': [3, 4, 5, 6, 8, 10],
        'learning_rate' : [0.005, 0.01, 0.02, 0.03, 0.05, 0.07],
        'tree_method' : ['auto', 'exact', 'approx']
        }

In [0]:
xgb = XGBClassifier()
# Random search of parameters, using default 5 fold cross validation, 
# search across different combinations, and use all available cores
xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = random_grid,
                                n_jobs=-1)
# Fit the random search model
xgb_random.fit(X_train, y_train)

In [0]:
#import XGBoost classifier and accuracy
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

learning_rate = xgb_random.best_params_["learning_rate"]
n_estimators = xgb_random.best_params_["n_estimators"]
max_depth = xgb_random.best_params_["max_depth"]
tree_method = xgb_random.best_params_["tree_method"]
booster = xgb_random.best_params_["booster"]

#instantiate model and train
model = XGBClassifier(learning_rate = learning_rate, n_estimators=n_estimators, max_depth=max_depth,
                       tree_method =tree_method, booster = booster, ,n_jobs=-1)
model.fit(X_train, y_train)

In [0]:
test_df = pd.read_csv("test.csv", index_col = "Loan_ID")

In [0]:
test_df.isna().sum()

In [0]:
test_df = cleaning_data(test_df)

In [0]:
test_df = label_encode(test_df)

In [0]:
scaled_features_test_df = normalize_data(test_df)

In [0]:
predictions = model.predict(scaled_features_test_df)

In [0]:
output_df = pd.DataFrame({'Loan_ID': scaled_features_test_df.index, 'Interest_Rate': predictions})

In [0]:
output_df.to_csv("solution.csv", index=False)

In [0]:
#from sklearn.metrics import f1_score

In [0]:
#f1_score(y_test, predictions, average='weighted')